### 1. Gérer les données avec *Dataset* et *Dataloader*


Les classes `Dataset` et `Dataloader` permettent de faciliter la gestion des données sous `PyTorch`. 

La classe `Dataset` est une classe abstraite qui permet de préciser comment un exemple est chargé, pré-traité, transformé ... et donne accès par l'intermédiaire d'un itérateur à chaque exemple d'un jeu de donnnées. 

La classe `Dataloader` encapsule un jeu de données et permet de requêter de diverses manières ce jeu de données: spécifier la taille du mini-batch, si l'ordre doit être aléatoire ou non, de quelle manière sont concaténés les exemples etc...

Pour implémenter un `Dataset`, il suffit de définit 2 méthodes `__getitem__(self, index)` et `__len(self)`:

`from torch.utils.data import Dataset, DataLoader`

`class MonDataset(Dataset):`

    `def __init__(self, ...):`

        `pass`

    `def __getitem__(self, index):`

    retourne un couple (exemple, label) correspondant à l'index
        
        `pass`
    
    `def __len__(self):`

    renvoie la taille du jeu de données
    
        `pass`
    

Principaux avantages (mis à part le prétraitement possible) est qu'il n'est pas nécessaire de charger tout le jeu de données en mémoire, le chargement se fait à la volée. Par ailleurs, la classe pré-existante `TensorDataset` permet de construire un dataset pour une liste de tenseurs passée en argument (le ième exemple est un nuplet composé de la ième ligne de chaque tenseur)

Une fois `MonDataset` implémenté il suffit de créer un `DataLoader` de la manière suivante :

`data = DataLoader(MonDataset(...), shuffle = True, batch_size = BATCH_SIZE)`

création du dataloader en spécifiant la taille du batch et ordre aléatoire

`for x, y in data:`

    x et y est un batch de taille BATCH_SIZE